# Info

This file is there to setup a connected database from all the individual scrabed and filtered files.

Setup database

In [1]:
from sqlalchemy import create_engine
from database_structure import Base
engine = create_engine(f"sqlite:///dorfterror.db")
Base.metadata.create_all(engine)

## Create Profiles and Follower table

In [2]:
import pandas as pd
from pathlib import Path

root_path = Path(".").resolve().parent

follower_df = pd.read_parquet(f"{root_path}/filtered_datasets/all_dorfterror_followers.parquet")
display(follower_df.head())

,id,fbid,profile_url,username,full_name,follower_count,followees_count,post_count,profile_image_link,biography,...,is_verified,is_private,avg_engagement,posts,highlight_count,partner_id,email_address,business_address,related_profiles,external_url
0,44961905641,17841444929240238,https://instagram.com/black_pitt26,black_pitt26,Black Pitt,74,345,39,https://instagram.fphl1-1.fna.fbcdn.net/v/t51....,None,...,False,True,NaN,None,0.0,4.496191e+10,None,None,[[]],None
1,420609147,17841400034416572,https://instagram.com/hardywest,hardywest,Hardy Westerhoff,882,2027,54,https://instagram.fopo2-2.fna.fbcdn.net/v/t51....,None,...,False,True,NaN,None,0.0,4.206091e+08,None,None,[[]],"[""http://www.facebook.de/dinnerfreunde""]"
2,74961695035,17841475091060212,https://instagram.com/derb.enz,derb.enz,Ben,82,1504,1,https://instagram.fopo3-1.fna.fbcdn.net/v/t51....,🎮 Gaming \n🖤🤘🏼Metal,...,False,True,NaN,None,0.0,7.496170e+10,None,None,[[]],None
3,26041726184,17841426105535216,https://instagram.com/carsten_cavelius,carsten_cavelius,CCA,280,1192,7,https://instagram.fath4-2.fna.fbcdn.net/v/t51....,None,...,False,True,0.1357,"[[{""caption"":""🟠 NEU in Merzig – Kunst zum Anzi...",0.0,2.604173e+10,None,None,[[]],None
4,3114860088,17841403002669356,https://instagram.com/lennard.d,lennard.d,Lennard Dirksmeier,144,282,0,https://instagram.fcai19-2.fna.fbcdn.net/v/t51...,None,...,False,True,NaN,None,0.0,3.114860e+09,None,None,[[]],None


In [3]:
from sqlalchemy.orm import sessionmaker
from database_structure import Profile, Follower
from formatting_utils import format_list

Session = sessionmaker(bind=engine)
session = Session()

for _, row in follower_df.iterrows():
    profile_id = str(row['id'])

    # Prüfen ob Profil schon existiert
    profile = session.query(Profile).get(profile_id)
    if not profile:
        profile = Profile(
            id=profile_id,
            fbid=str(row['fbid']),
            username=row['username'],
            full_name=row['full_name'],
            profile_url=row['profile_url'],
            follower_count=int(row['follower_count']),
            followees_count=int(row['followees_count']),
            post_count=int(row['post_count']),
            profile_image_link=row['profile_image_link'],
            biography=row['biography'],
            biography_hashtags=format_list(row['biography_hashtags']),
            category_name=row['category_name'],
            business_category_name=row['business_category_name'],
            is_business_account=row['is_business_account'],
            is_professional_account=row['is_professional_account'],
            is_verified=row['is_verified'],
            is_private=row['is_private'],
            avg_engagement=row['avg_engagement'],
            posts=format_list(row['posts']),
            highlight_count=row['highlight_count'],
            partner_id=row['partner_id'],
            email_address=row['email_address'],
            business_address=row['business_address'],
            related_profiles=format_list(row['related_profiles']),
            external_url=row['external_url'],
        )
        session.add(profile)

    follower = Follower(
        profile_id=profile_id,
        username=row['username'],
    )
    session.add(follower)

session.commit()
session.close()


/tmp/ipykernel_13761/3203852503.py:12: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  profile = session.query(Profile).get(profile_id)


### Add Dorfterror to Profile Table

In [4]:
import pandas as pd
from pathlib import Path

root_path = Path(".").resolve().parent

dorfterror_profile_df = pd.read_csv(f"{root_path}/scraped_data/dorfterror_profile_info.csv")
display(dorfterror_profile_df.head())


,account,fbid,id,followers,posts_count,is_business_account,is_professional_account,is_verified,avg_engagement,external_url,...,partner_id,business_address,related_accounts,email_address,timestamp,input,error,error_code,warning,warning_code
0,dorfterror,17841406991784974,7079793059,4204,562,True,True,False,0.0336,"[""https://hyperfollow.com/dorfterror"",""https:/...",...,7079793059,NaN,"[{""id"":""1394525203"",""profile_name"":""Kingstar M...",booking@dorfterror.de,2025-07-09T18:14:02.030Z,"{""url"":""https://www.instagram.com/dorfterror/""}",NaN,NaN,NaN,NaN


In [5]:
from sqlalchemy.orm import sessionmaker
from database_structure import Profile
from formatting_utils import format_list

Session = sessionmaker(bind=engine)
session = Session()

profile_id = dorfterror_profile_df['id'].iloc[0]

# Prüfen ob Profil schon existiert
profile = session.query(Profile).get(profile_id)
if not profile:
    profile = Profile(
        id=str(profile_id),
        fbid=str(dorfterror_profile_df['fbid'].iloc[0]),
        username=dorfterror_profile_df['account'].iloc[0],
        full_name=dorfterror_profile_df['full_name'].iloc[0],
        profile_url=dorfterror_profile_df['url'].iloc[0],
        follower_count=int(dorfterror_profile_df['followers'].iloc[0]),
        followees_count=int(dorfterror_profile_df['following'].iloc[0]),
        post_count=int(dorfterror_profile_df['posts_count'].iloc[0]),
        profile_image_link=dorfterror_profile_df['profile_image_link'].iloc[0],
        biography=dorfterror_profile_df['biography'].iloc[0],
        biography_hashtags=format_list(dorfterror_profile_df['bio_hashtags'].iloc[0]),
        category_name=dorfterror_profile_df['category_name'].iloc[0],
        business_category_name=dorfterror_profile_df['business_category_name'].iloc[0],
        is_business_account=dorfterror_profile_df['is_business_account'].iloc[0],
        is_professional_account=dorfterror_profile_df['is_professional_account'].iloc[0],
        is_verified=dorfterror_profile_df['is_verified'].iloc[0],
        is_private=dorfterror_profile_df['is_private'].iloc[0],
        avg_engagement=dorfterror_profile_df['avg_engagement'].iloc[0],
        posts=format_list(dorfterror_profile_df['posts'].iloc[0]),
        highlight_count=dorfterror_profile_df['highlights_count'].iloc[0],
        partner_id=dorfterror_profile_df['partner_id'].iloc[0],
        email_address=dorfterror_profile_df['email_address'].iloc[0],
        business_address=dorfterror_profile_df['business_address'].iloc[0],
        related_profiles=format_list(dorfterror_profile_df['related_accounts'].iloc[0]),
        external_url=dorfterror_profile_df['external_url'].iloc[0],
    )
    session.add(profile)

session.commit()
session.close()


/tmp/ipykernel_13761/924126990.py:11: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  profile = session.query(Profile).get(profile_id)


### Add non following accounts to Profile Table

In [6]:
import pandas as pd
from pathlib import Path

root_path = Path(".").resolve().parent

non_following_profiles_df = pd.read_parquet(f"{root_path}/filtered_datasets/all_dorfterror_non_follower_profiles.parquet")
display(non_following_profiles_df.head())


,id,fbid,profile_url,username,full_name,follower_count,followees_count,post_count,profile_image_link,biography,...,is_verified,is_private,avg_engagement,posts,highlight_count,partner_id,email_address,business_address,related_profiles,external_url
0,21547484656,17841421499703435,https://instagram.com/deutsche.asphalt,deutsche.asphalt,None,399,395,15,https://instagram.fbcn7-3.fna.fbcdn.net/v/t51....,None,...,False,True,NaN,None,0,21547484656,None,None,[],None
1,2042492691,17841402060745311,https://instagram.com/merlin_b3,merlin_b3,✟ 🇩🇪,319,329,0,https://scontent-nrt1-1.cdninstagram.com/v/t51...,𝕯𝖚𝖎𝖘𝖇𝖚𝖗𝖌\n𝕬𝖛𝖊 𝕮𝖍𝖗𝖎𝖘𝖙𝖚𝖘 𝕽𝖊𝖝,...,False,True,NaN,None,0,2042492691,None,None,[],None
2,5775462240,17841405947201932,https://instagram.com/joeli_069,joeli_069,joeli_069,542,1199,21,https://instagram.fomr1-1.fna.fbcdn.net/v/t51....,@vcfrankfurt x @lifeisaride.team x @flamme_rou...,...,False,True,NaN,None,0,5775462240,None,None,[],None
3,48032082400,17841448072004010,https://instagram.com/kryszian,kryszian,Christian Steuer,246,265,72,https://instagram.fdub5-2.fna.fbcdn.net/v/t51....,..früher war mehr Lametta..,...,False,False,0.2757,"[{""comments"":2,""datetime"":""2025-06-15T01:04:40...",3,48032082400,None,None,[],None
4,7490750662,17841407639866343,https://instagram.com/karin.thome.anton,karin.thome.anton,Karin Thomé,295,942,75,https://scontent-waw2-2.cdninstagram.com/v/t51...,None,...,False,True,NaN,None,0,7490750662,None,None,[],None


In [7]:
from sqlalchemy.orm import sessionmaker
from database_structure import Profile, Follower
from formatting_utils import format_list

Session = sessionmaker(bind=engine)
session = Session()

for _, row in non_following_profiles_df.iterrows():
    profile_id = str(row['id'])

    # Prüfen ob Profil schon existiert
    profile = session.query(Profile).get(profile_id)
    if not profile:
        profile = Profile(
            id=profile_id,
            fbid=str(row['fbid']),
            username=row['username'],
            full_name=row['full_name'],
            profile_url=row['profile_url'],
            follower_count=int(row['follower_count']),
            followees_count=int(row['followees_count']),
            post_count=int(row['post_count']),
            profile_image_link=row['profile_image_link'],
            biography=row['biography'],
            biography_hashtags=format_list(row['biography_hashtags']),
            category_name=row['category_name'],
            business_category_name=row['business_category_name'],
            is_business_account=row['is_business_account'],
            is_professional_account=row['is_professional_account'],
            is_verified=row['is_verified'],
            is_private=row['is_private'],
            avg_engagement=row['avg_engagement'],
            posts=format_list(row['posts']),
            highlight_count=row['highlight_count'],
            partner_id=row['partner_id'],
            email_address=row['email_address'],
            business_address=row['business_address'],
            related_profiles=format_list(row['related_profiles']),
            external_url=row['external_url'],
        )
        session.add(profile)

session.commit()
session.close()

/tmp/ipykernel_13761/691991597.py:12: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  profile = session.query(Profile).get(profile_id)


## Create post table

In [8]:
import pandas as pd
from pathlib import Path

root_path = Path(".").resolve().parent

post_df = pd.read_parquet(f"{root_path}/filtered_datasets/all_dorfterror_posts.parquet")
category_df = pd.read_parquet(f"{root_path}/filtered_datasets/posts_categories.parquet")

category_df["post_id"] = category_df["post_id"].astype(int)
category_df["category"] = category_df["category"].astype(str)


post_df = post_df.merge(category_df, on='post_id', how='left')

display(post_df.head())

,url,user_posted,description,hashtags,num_comments,date_posted,likes,photos,videos,location,...,is_paid_partnership,partnership_details,user_posted_id,post_content,audio,videos_duration,images,alt_text,timestamp,category
0,https://www.instagram.com/p/DHizfkMMTF7,dorfterror,"Kölle, dao bis asozial! \n\nTausend Dank an eu...","[""#dorfterror"",""#nofuturekid"",""#köln""]",3,2025-03-23 13:38:35+00:00,195.0,"[""https://instagram.fagc3-2.fna.fbcdn.net/v/t5...",None,"[""Club Castell""]",...,False,"{""profile_id"":null,""username"":null,""profile_ur...",7079793059,"[{""index"":0,""type"":""Photo"",""url"":""https://inst...","{""audio_asset_id"":null,""original_audio_title"":...",[],"[{""url"":""https://instagram.fagc3-2.fna.fbcdn.n...","Photo shared by Dorfterror on March 23, 2025 t...",2025-07-12T09:42:56.049Z,Live & Tour
1,https://www.instagram.com/p/DHdl8fAM6DV,dorfterror,Was geht denn jetzt ab??! 🤩\n\nWir sind dieses...,None,16,2025-03-21 13:04:00+00:00,129.0,"[""https://scontent-ord5-3.cdninstagram.com/v/t...",None,None,...,False,"{""profile_id"":null,""username"":null,""profile_ur...",7079793059,"[{""index"":0,""type"":""Photo"",""url"":""https://scon...","{""audio_asset_id"":null,""original_audio_title"":...",[],"[{""url"":""https://scontent-ord5-3.cdninstagram....","Photo shared by Dorfterror on March 21, 2025 t...",2025-07-12T09:42:58.405Z,Live & Tour
2,https://www.instagram.com/reel/DHdWpa5su0n,muellem_mon_amour.161,@minohband \n@dorfterror \n@castell\nRegentens...,None,0,2025-03-21 10:55:42+00:00,51.0,"[""https://scontent-iad3-1.cdninstagram.com/v/t...","[""https://scontent-iad3-2.cdninstagram.com/o1/...","[""Club Castell""]",...,False,"{""profile_id"":null,""username"":null,""profile_ur...",6196585073,"[{""index"":0,""type"":""Video"",""url"":""https://scon...","{""audio_asset_id"":null,""original_audio_title"":...","[{""url"":""https://scontent-iad3-2.cdninstagram....",[],None,2025-07-12T09:42:59.888Z,Live & Tour
3,https://www.instagram.com/p/DHdOnEzM7SO,dorfterror,Klingeling! Heute Abend spielen wir in #köln u...,"[""#köln"",""#punkrock"",""#keinbockaufnazis"",""#dor...",4,2025-03-21 09:40:06+00:00,127.0,"[""https://scontent-atl3-1.cdninstagram.com/v/t...",None,"[""Club Castell""]",...,False,"{""profile_id"":null,""username"":null,""profile_ur...",7079793059,"[{""index"":0,""type"":""Photo"",""url"":""https://scon...","{""audio_asset_id"":null,""original_audio_title"":...",[],"[{""url"":""https://scontent-atl3-1.cdninstagram....","Photo shared by Dorfterror on March 21, 2025 t...",2025-07-12T09:43:03.772Z,Live & Tour
4,https://www.instagram.com/p/DHWUm_jsm7c,dorfterror,Am Samstag geht es für uns nach Klein Ammensle...,"[""#scheune""]",0,2025-03-18 17:17:50+00:00,76.0,"[""https://scontent-lga3-2.cdninstagram.com/v/t...",None,"[""Klein Ammensleben"",""Sachsen-Anhalt"",""Germany""]",...,False,"{""profile_id"":null,""username"":null,""profile_ur...",7079793059,"[{""index"":0,""type"":""Photo"",""url"":""https://scon...","{""audio_asset_id"":null,""original_audio_title"":...",None,[],"Photo by Dorfterror in Klein Ammensleben, Sach...",2025-07-12T09:43:04.786Z,Live & Tour


In [9]:
from sqlalchemy.orm import sessionmaker
from database_structure import Post, Collaborator
from formatting_utils import format_list, extract_mentions, get_media_count, get_video_duration, get_total_follower_reach

Session = sessionmaker(bind=engine)
session = Session()

all_profiles = session.query(Profile).all()
profile_map = {profile.username.lower(): (profile.id, profile.follower_count) for profile in all_profiles}

for _, row in post_df.iterrows():
    media_count = get_media_count(row['post_content'])
    video_duration = get_video_duration(row['videos_duration'])
    post = Post(
        id = str(row['post_id']),
        shortcode = row['shortcode'],
        date_local = row['date_posted'],
        url = row['url'],
        user_posted = row['user_posted'],
        content_type = row['content_type'],
        caption = row['description'],
        caption_hashtags = format_list(row['hashtags']),
        caption_mentions = extract_mentions(row['description']),
        tagged_users = format_list(row['tagged_users']),
        video_view_count = row['video_view_count'],
        video_play_count = row['video_play_count'],
        video_duration = video_duration,
        location = format_list(row['location']),
        media_count = media_count,
        post_content = format_list(row['post_content']),
        photos = format_list(row['photos']),
        videos = format_list(row['videos']),
        audio = format_list(row['audio']),
        images = format_list(row['images']),
        alt_text = row['alt_text'],
        thumbnail = row['thumbnail'],
        product_type= row['product_type'],
        coauthor_producers = format_list(row['coauthor_producers']),
        engagement_score_view= row['engagement_score_view'],
        like_count = row['likes'],
        comment_count = row['num_comments'],
        is_collab = True if row['coauthor_producers'] is not None else False,
        total_follower_reach = get_total_follower_reach(user_post=row['user_posted'], collaborators=format_list(row['coauthor_producers']), profile_mapping=profile_map),
        category = row['category']
    )
    session.add(post)

    if row['coauthor_producers'] is not None and row['coauthor_producers'] != []:
        all_connected_accounts = set(format_list(row['coauthor_producers']) + [row['user_posted']])
        all_collaborators = all_connected_accounts - {"dorfterror"}
        for current_collaborator in all_collaborators:
            profile_id, _ = profile_map.get(current_collaborator, (None, None))
            collaborator = Collaborator(
                post_id = int(row['post_id']),
                profile_id=profile_id,
                username=current_collaborator
            )
            session.add(collaborator)
session.commit()
session.close()

## Create comments table

In [10]:
import pandas as pd
from pathlib import Path

root_path = Path(".").resolve().parent

comment_df = pd.read_parquet(f"{root_path}/filtered_datasets/all_dorfterror_comments.parquet")
print(comment_df.shape)
display(comment_df.head())

(1728, 14)


,comment_user,comment_user_url,comment_date,comment,likes_number,replies_number,replies,hashtag_comment,tagged_users_in_comment,post_url,comment_id,post_id,timestamp,parent_comment_id
0,rambow_band,https://www.instagram.com/rambow_band,2024-11-05T11:43:12.000Z,🤘🏻😎,1,0.0,None,None,None,https://www.instagram.com/reel/DB6rfmRMaex,18135107722369506,3493295748373653425,2025-07-12T20:59:16.559Z,NaN
1,its_a_161_riot,https://www.instagram.com/its_a_161_riot,2024-11-03T21:34:50.000Z,🔥🔥🔥,1,0.0,None,None,None,https://www.instagram.com/reel/DB6rfmRMaex,18062444656780999,3493295748373653425,2025-07-12T20:59:16.559Z,NaN
2,dorfterror,https://www.instagram.com/dorfterror,2024-11-03T17:32:54.000Z,"Auch wenn du aus Wuppertal kommst, bist du tro...",6,1.0,"[{""reply_user"":""ichbineinfynn"",""reply_date"":""2...",None,None,https://www.instagram.com/reel/DB6rfmRMaex,18061673317749633,3493295748373653425,2025-07-12T20:59:16.559Z,NaN
3,daswoerm,https://www.instagram.com/daswoerm,2025-03-17T17:06:33.000Z,Es war uns ein Fest! Gerne wieder 🙌,1,1.0,"[{""reply_user"":""dorfterror"",""reply_date"":""2025...",None,None,https://www.instagram.com/p/DHTmzkhsJFX,18098628496518632,3590383997625078103,2025-07-12T20:59:16.584Z,NaN
4,johnnyraw_support,https://www.instagram.com/johnnyraw_support,2025-03-17T16:59:28.000Z,Danke für dieses Tolle Erlebnis.\nIch hoffe wi...,3,1.0,"[{""reply_user"":""dorfterror"",""reply_date"":""2025...",None,None,https://www.instagram.com/p/DHTmzkhsJFX,18088934050570684,3590383997625078103,2025-07-12T20:59:16.584Z,NaN


In [11]:
from sqlalchemy.orm import sessionmaker
from database_structure import Comment
from formatting_utils import format_list, did_dorfterror_reply

Session = sessionmaker(bind=engine)
session = Session()
profiles = session.query(Profile).all()
profile_map = {
    profile.username.lower(): (profile.id)
    for profile in profiles
}
for _, row in comment_df.iterrows():
    comment = Comment(
        id = str(row['comment_id']),
        post_id = str(row['post_id']),
        profile_id = profile_map.get(row['comment_user'].lower(), None),
        user_name = row['comment_user'],
        parent_comment_id = row['parent_comment_id'],
        post_url = row['post_url'],
        replies_count = row['replies_number'],
        replied_content = format_list(row['replies']),
        text = row['comment'],
        likes_count = row['likes_number'],
        created_at = pd.to_datetime(row['comment_date']),
        hashtags = format_list(row['hashtag_comment']),
        tags = format_list(row['tagged_users_in_comment']),
        creator_replied = did_dorfterror_reply(format_list(row['replies'])),
    )
    session.add(comment)
session.commit()
session.close()

## Calculate all missing values

Calculate comment_count for each follower

In [12]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import func
from database_structure import Follower, Comment

Session = sessionmaker(bind=engine)
session = Session()

comment_counts = (
    session.query(Comment.profile_id, func.count(Comment.id))
    .join(Follower, Comment.profile_id == Follower.profile_id)
    .group_by(Comment.profile_id)
    .all()
)

comment_count_map = {profile_id: count for profile_id, count in comment_counts}

followers = session.query(Follower).all()
for follower in followers:
    follower.comment_count = comment_count_map.get(follower.profile_id, 0)

session.commit()
session.close()
